In [1]:
import os
import importlib
import numpy as np
import yaml
from tqdm.auto import tqdm
import seesaw
import ray 
import seesaw.seesaw_bench
import seesaw.figures
import seesaw.dataset_manager
import seesaw.dataset

importlib.reload(seesaw.metrics)
importlib.reload(seesaw.figures)
importlib.reload(seesaw.seesaw_bench)

import seesaw.figure_utils
importlib.reload(seesaw.figure_utils)
from seesaw.figure_utils import *
from seesaw.seesaw_bench import *
from seesaw.figures import *
from plotnine import *
from seesaw.dataset_manager import GlobalDataManager


In [2]:
#ray.init('auto', namespace='seesaw')

In [3]:
import glob

In [44]:
results_paths = [    
    # ('', '//home/gridsan/omoll/test_new_trainer/bench_DHeCZhAeux/'),    
    # ('', '//home/gridsan/omoll/test_new_trainer/bench_LAaCMfitvl/'), #'baseline'
    # ('', '//home/gridsan/omoll/test_new_trainer/bench_jYTXlGTLdu/'), # rocchio hyperparams
    # ('', './bench_NrMYlLXfJo/'), # seesaw hyperparams 1
    # ('', './bench_AEZJiQmHaT/'), # seesaw hyperparams 2
    # ('', './bench_jYTXlGTLdu/'), # rocchio hyperparams
    # ('', './bench_qjarjqiTtP/'),
#    ('', './bench_jPIfrrviVd/'),
    # ('', './bench_LKbANKUZXD/'), # latency
    # ('', './bench_UIdAAkpcRg/'), # latency
    # ('', './bench_igTgtspZYb/'), # latency
    ('', './bench_HVOsShtClv'), # rocchio coarse
    ('', './bench_cUQkrzpYaR/') # few shot coarse
    #(/summary.parquet
]

## are the results between current version and previous version really identical? unlikely...
## next steps: 
# 1. check something changed for some dataset before and after.
# if yes, then nothing to do.
# or check if the lambda is perhaps too high.

#all_summs = [checkpoint]
all_summs = []
for (i,(nm, bench_folder)) in enumerate(results_paths):
    summss = get_all_session_summaries(bench_folder, force_recompute=False, parallel=False)
    summss = summss.assign(bench_folder=bench_folder, bench_id=bench_folder.rstrip('/').rsplit('/')[-1])
    summss = summss.assign(run_id = summss.bench_id + ('_' + summss.sample_id).fillna(''), timestamp=get_timestamp(bench_folder))
    
    
    if nm != '':
        summss = summss.assign(name=summss.name + '_' + nm)
        
    all_summs.append(summss)
    
summs = pd.concat(all_summs, ignore_index=True).reset_index(drop=True)
#summs.to_parquet('/home/gridsan/omoll/concat_summs_checkpoint_Mar1.parquet')

In [85]:
#summs['latencies'].map(np.mean)

In [38]:
### table with results vs. baselines.
### table with hyperparam results for both rocchio, seesaw.

In [240]:
size_df = pd.DataFrame.from_records([
    ##
    {'dataset':'bdd', 'index_name':'multiscalecoarse', 'vectors':79863},
    {'dataset':'bdd', 'index_name':'multiscalemed', 'vectors':1597260},
    ##
    {'dataset':'coco', 'index_name':'multiscalecoarse', 'vectors':119979},
    {'dataset':'coco', 'index_name':'multiscalemed', 'vectors':1614866},
    ##
    {'dataset':'objectnet', 'index_name':'multiscalecoarse', 'vectors':50273},
    {'dataset':'objectnet', 'index_name':'multiscalemed', 'vectors':50273}    
])

size_df

,dataset,index_name,vectors
0,bdd,multiscalecoarse,79863
1,bdd,multiscalemed,1597260
2,coco,multiscalecoarse,119979
3,coco,multiscalemed,1614866
4,objectnet,multiscalecoarse,50273
5,objectnet,multiscalemed,50273


In [230]:
def human_format(num):
    num = float('{:.2g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])

In [235]:
def table_dataset_name(dataset, index):
    if index == 'multiscalemed':
        return dataset
    elif index == 'multiscalecoarse':
        return dataset + '$^-$'
    else:
        assert False

In [45]:
bdd_ignore = ['other vehicle', 'side traffic light', 
              'green traffic light', 'yellow traffic light', 'red traffic light', 
              'dawn or dusk', 'daytime', 'night']

stats = summs.assign(**post_process(summs))
stats = stats.assign(session_params_dict=stats.session_params.map(lambda p : json.loads(p)))
stats = stats.assign(start_policy=stats.session_params_dict.map(lambda x : x['start_policy']))

stats = stats.assign(latency_mean=stats.latencies.map(np.mean))
stats = stats.assign(latency_stderr=stats.latencies.map(np.std))

stats = stats.assign(variant=stats.variant.map(lambda name : {'baseline':'CLIP', 'multi_reg':'\sys/', 
                                                              'active_search':'ENS', 'pseudo_lr_switch':'propagation', 
                                                              'rocchio_update':'Rocchio' }.get(name, name)))

# avgs = (stats[['dataset', 'index_name', 'variant', 'latency_mean']]
#         .groupby(['dataset', 'index_name', 'variant']).latency_mean.mean())

# avgs2 = avgs.unstack().reset_index()

# df = size_df.merge(avgs2,left_on=['dataset', 'index_name'], right_on=['dataset', 'index_name'], how='right')
# df = df.assign(dataset=df.dataset.map(lambda name : {'objectnet':'ObjNet',
#                     'bdd':'BDD',
#                     'coco':'COCO/LVIS'}.get(name,None)))
# df.insert(loc=0, column='dsplus', value=df[['dataset','index_name']].apply(tuple, axis=1).map(lambda tup: table_dataset_name(tup[0], tup[1])))
# df = df.sort_values(['vectors'])

# print(df.drop(['dataset', 'index_name'], axis=1)
#       .style
#       .format({'vectors':human_format},precision=2).hide(axis=0)
#       .to_latex(column_format='l|r|rrrrr',
#                 label=f'tab:latency',hrules=True,
#                 clines='all;index', 
#                 caption=f'Iteration latency (in seconds) vs dataset scales (in vectors). $^-$ means coarse indexing (one vector per image)'))

/Users/orm/mambaforge/envs/seesaw/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
/Users/orm/mambaforge/envs/seesaw/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
/Users/orm/mambaforge/envs/seesaw/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
/Users/orm/mambaforge/envs/seesaw/lib/python3.10/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
/Users/orm/mambaforge/envs/seesaw/lib/python3.10/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide


In [46]:


# ds.path='/data1/groups/fastai/omoll/seesaw_root2/data/bdd' ds.paths.shape[0]= p.index_spec.i_name='multiscalemed' idx.vectors.shape[0]=1597260
# ds.path='/data1/groups/fastai/omoll/seesaw_root2/data/coco' ds.paths.shape[0]= p.index_spec.i_name='multiscalemed' idx.vectors.shape[0]=1614866
# ds.path='/data1/groups/fastai/omoll/seesaw_root2/data/objectnet' ds.paths.shape[0]=50273 p.index_spec.i_name='multiscalemed' idx.vectors.shape[0]=

In [47]:
#stats.session_params_dict.iloc[-1]

In [48]:
ps = (stats[['session_params_dict']]
    .apply(apply_funs(get_active_search_params2, 
                      get_matrix_params, 
                      get_multi_reg_params,
                        get_rocchio_params), axis=1, result_type='expand'))
stats = stats.assign(**ps)

stats = stats[stats.category.map(lambda x : x not in bdd_ignore and not x.endswith('weather') and not x.endswith('scene'))]

subset = stats[stats.index_name.isin(['multiscalecoarse', 'multiscalemed']) 
                    &  stats.reg_norm_lambda.isin([np.nan, 100.]) 
                    & stats.variant.isin(['baseline', 'multi_reg', 'active_search', 'rocchio_update'])] 

In [49]:
old_eval_data = pd.read_parquet('../figures/eval_data.parquet')
baseline_df = old_eval_data[old_eval_data.full_hash.eq('2501cb92_multiscalecoarse')]

#subset = pd.read_parquet('./rocchio_hyperparam.parquet')
subset = pd.concat([stats, baseline_df])

In [50]:
def add_hard_queries(eval_data):
    baseline_df = eval_data[eval_data.full_hash.eq('2501cb92_multiscalecoarse')]
    hard_baseline_df = baseline_df[baseline_df.average_precision < .5]
    hard_queries_df = hard_baseline_df[['dataset', 'category']].sort_values(['dataset', 'category']).reset_index(drop=True)
    eval_hard_df = eval_data.merge(hard_queries_df, left_on=('dataset', 'category'), right_on=('dataset', 'category'), how='right')

    eval_hard_df = eval_hard_df.assign(query_group='hard subset')
    eval_df = eval_data.assign(query_group='all queries')
    
    full_df = pd.concat([eval_df, eval_hard_df], ignore_index=True).reset_index(drop=True)
    full_df = full_df.assign(**{'query group':pd.Categorical(full_df.query_group, categories=['all queries', 'hard subset'], ordered=True)})
    
    return full_df

In [51]:
subset = add_hard_queries(subset)

In [52]:
subset.query_group.unique()

array(['all queries', 'hard subset'], dtype=object)

In [53]:
base_cols =  ['param_hash', 'index_name',  'timestamp', 'dataset', 'category','query_group']
detail_cols = ['full_hash', 'start_policy', 'variant', 'calibration', 'reward_horizon', 'pos_weight', 'reg_data_lambda', 'reg_query_lambda', 'reg_norm_lambda',
                          'rocchio_beta', 'rocchio_gamma']
metric_cols = ['reciprocal_rank', 'ndcg_score', 'average_precision']

In [54]:
eval_data = subset[base_cols + detail_cols + metric_cols]

In [62]:
s = display_means(eval_data,  metric='average_precision', other_cols=['query_group', 'index_name', 'variant',   'reg_query_lambda', 'reg_data_lambda', 'reg_norm_lambda', 'rocchio_beta', 'rocchio_gamma'], return_totals=True)

dataset,query_group,index_name,variant,reg_query_lambda,reg_data_lambda,reg_norm_lambda,rocchio_beta,rocchio_gamma,param_hash,timestamp,bdd,coco,lvis,objectnet,average
0,all queries,multiscalecoarse,Rocchio,nan,nan,nan,0.50,0.25,c45449fc,2023-07-20 21:43:03.560166,0.75,0.93,0.68,0.70,0.76
1,all queries,multiscalecoarse,\sys/,0.00,0.00,100.00,nan,nan,799bf389,2023-07-21 02:24:56.023244,0.73,0.88,0.65,0.58,0.71
2,all queries,multiscalecoarse,baseline,nan,nan,nan,nan,nan,2501cb92,2023-03-15 17:42:10,0.74,0.90,0.63,0.64,0.72
3,hard subset,multiscalecoarse,Rocchio,nan,nan,nan,0.50,0.25,c45449fc,2023-07-20 21:43:03.560166,0.05,0.49,0.28,0.38,0.30
4,hard subset,multiscalecoarse,\sys/,0.00,0.00,100.00,nan,nan,799bf389,2023-07-21 02:24:56.023244,0.06,0.32,0.25,0.28,0.23
5,hard subset,multiscalecoarse,baseline,nan,nan,nan,nan,nan,2501cb92,2023-03-15 17:42:10,0.02,0.27,0.19,0.28,0.19


In [63]:
tabs = s.drop([ 'index_name', 'timestamp', 'param_hash'], axis=1)
tabs = tabs[tabs.variant.isin(['baseline', 'Rocchio', '\sys/'])]

# gp = tabs.groupby(['query_group','variant', ])
#     gp = gp.dropna(axis=1)
   # gp = gp.drop(['variant', 'query_group'], axis=1)
gp = tabs.rename({'reg_norm_lambda':'$\\lambda$',
                'reg_query_lambda':'$\\lambda_q$',
                'reg_data_lambda':'$\\lambda_D$',   
                'rocchio_beta':'$\\beta$',
               'rocchio_gamma':'$\\gamma$',
               'lvis':'LVIS',
                'objectnet':'ObjNet',
                'average':'Avg.',
                'bdd':'BDD',
                'coco':'COCO',
               }, axis=1)

#    gp.assign(variant=gp.variant.map(lambda v : {'multi_reg':'this work', 'rocchio_update':'Rocchio'}.get(v,v))
str = (gp[['query_group', 'variant', 'LVIS', 'ObjNet','COCO', 'BDD', 'Avg.']].style
#           .hide(axis=0)
           .format(formatter={'$\\lambda$':'{:.0f}', '$\\lambda_q$':'{:.0f}', '$\\lambda_D$':'{:.0f}'}, precision=2)
          # .highlight_max(axis=0, subset=['LVIS', 'ObjNet','COCO', 'BDD', 'Avg.'], props='bfseries: ;')
           .to_latex(label=f'tab:hyperparam',hrules=True,
                     clines='all;index', caption=f'hyperparamters vs AP'))
print(str)

\begin{table}
\caption{hyperparamters vs AP}
\label{tab:hyperparam}
\begin{tabular}{lllrrrrr}
\toprule
dataset & query_group & variant & LVIS & ObjNet & COCO & BDD & Avg. \\
\midrule
0 & all queries & Rocchio & 0.68 & 0.70 & 0.93 & 0.75 & 0.76 \\
\cline{1-1}
1 & all queries & \sys/ & 0.65 & 0.58 & 0.88 & 0.73 & 0.71 \\
\cline{1-1}
2 & all queries & baseline & 0.63 & 0.64 & 0.90 & 0.74 & 0.72 \\
\cline{1-1}
3 & hard subset & Rocchio & 0.28 & 0.38 & 0.49 & 0.05 & 0.30 \\
\cline{1-1}
4 & hard subset & \sys/ & 0.25 & 0.28 & 0.32 & 0.06 & 0.23 \\
\cline{1-1}
5 & hard subset & baseline & 0.19 & 0.28 & 0.27 & 0.02 & 0.19 \\
\cline{1-1}
\bottomrule
\end{tabular}
\end{table}



In [61]:
# summary table
summ = s[s.param_hash.isin(['583a1edc', 'c45449fc'])]

In [62]:
summ = summ[['query_group', 'variant', 'bdd', 'coco','lvis','objectnet','average']]
summ = summ.rename({'reg_norm_lambda':'$\\lambda$',
                    'reg_query_lambda':'$\\lambda_q$',
                    'reg_data_lambda':'$\\lambda_D$',   
                    'rocchio_beta':'$\\beta$',
                   'rocchio_gamma':'$\\gamma$',
                   'lvis':'LVIS',
                    'objectnet':'ObjNet',
                    'average':'Avg.',
                    'bdd':'BDD',
                    'coco':'COCO',
                    'query_group':'query group',
                   }, axis=1)


In [63]:
summ = summ.assign(variant=summ.variant.map(lambda nm : {'multi_reg':'this work', 'rocchio_update':'Rocchio'}.get(nm,nm)))

In [64]:
str = (summ.style
               .hide(axis=0)
               .format(precision=2)
               .highlight_max(axis=0, subset=['BDD', 'COCO', 'LVIS', 'ObjNet', 'Avg.'], props='bfseries: ;')
               .to_latex(label=f'tab:hyperparam_{k}',hrules=True,
                         clines='all;index', caption=f'sensitivity to hyperparamters for method:{k}'))
print(str)

\begin{table}
\caption{sensitivity to hyperparamters for method:rocchio_update}
\label{tab:hyperparam_rocchio_update}
\begin{tabular}{llrrrrr}
\toprule
query group & variant & BDD & COCO & LVIS & ObjNet & Avg. \\
\midrule
all queries & this work & \bfseries 0.79 & 0.96 & \bfseries 0.76 & \bfseries 0.70 & \bfseries 0.80 \\
all queries & Rocchio & 0.75 & \bfseries 0.97 & 0.74 & 0.70 & 0.79 \\
hard subset & this work & 0.24 & 0.75 & 0.44 & 0.40 & 0.46 \\
hard subset & Rocchio & 0.09 & 0.86 & 0.40 & 0.38 & 0.43 \\
\bottomrule
\end{tabular}
\end{table}

